In [6]:
import datetime
!pip install numpy
!pip install pandas
!pip install tables
import pandas as pd
!pip install tqdm
from tqdm import tqdm
import requests
import os
from dotenv import load_dotenv
import json
import datetime

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [7]:
df = pd.read_pickle('processed_data.pkl')
training_df = df[df["train"]]
testing_df = df[df["test"]]

In [8]:
df

,file_id,user_id,subforum_id,num_contexts,hate,text,train,test
0,12834217_1,572066,1346,0,False,"As of March 13th , 2014 , the booklet had been...",False,False
1,12834217_2,572066,1346,0,False,In order to help increase the booklets downloa...,True,False
2,12834217_3,572066,1346,0,False,( Simply copy and paste the following text int...,False,False
3,12834217_4,572066,1346,0,True,Click below for a FREE download of a colorfull...,True,False
4,12834217_5,572066,1346,0,False,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,False,False
...,...,...,...,...,...,...,...,...
10939,33676864_5,734541,1388,0,False,"Billy - `` That guy would n't leave me alone ,...",False,False
10940,33677019_1,735154,1388,0,False,Wish we at least had a Marine Le Pen to vote f...,False,False
10941,33677019_2,735154,1388,0,False,Its like the choices are white genocide candid...,False,False
10942,33677053_1,572266,1388,0,True,Why White people used to say that sex was a si...,True,False


In [9]:
file_names = ['prompt-variations/v0.txt', 'prompt-variations/v1.txt']
SYSTEM_PROMPT = []
for file_name in file_names:
    with open(file_name, 'r') as file:
        SYSTEM_PROMPT.append(file.read())

In [20]:
def run_model(prompt_id, model):
    valid_models = ['codellama-34b-instruct', 'llama-2-70b-chat', 'mistral-7b-instruct', 'mixtral-8x7b-instruct', 'pplx-7b-chat', 'pplx-70b-chat']

    if model not in valid_models:
        raise ValueError(f'Invalid model {model}. Valid options are {valid_models}')

    load_dotenv() 
    PERPLEXITY_API_KEY = os.getenv("PPLX_KEY")

    results = []
    for index, row in tqdm(testing_df.iterrows(), total=testing_df["text"].count()):
        payload = {
            "model": model,
            "messages": [
                {
                    "role": "system",
                    "content": SYSTEM_PROMPT[prompt_id]
                },
                {
                    "role": "user",
                    "content": row["text"]
                }
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization": f"Bearer {PERPLEXITY_API_KEY}"
        }

        response = requests.post("https://api.perplexity.ai/chat/completions", json=payload, headers=headers)
        data = json.loads(response.text)
        answer = data['choices'][0]['message']['content']
        results.append({
            "model": model,
            "time": datetime.datetime.now(),
            "prompt": SYSTEM_PROMPT[prompt_number],
            "text": row["text"],
            "answer": answer,
            "truth": row["hate"]
        })
        print(row["text"])
        if lines := [line for line in answer.split("\n") if "hate_speech_probability" in line]:
            print(lines)
        else:
            print(answer)

    pd.DataFrame(results).to_pickle("perplexity_results.pkl")

In [21]:
results

[{'model': 'pplx-7b-chat',
  'time': datetime.datetime(2024, 1, 9, 17, 2, 31, 872701),
  'prompt': ['You are Sally and your job is to be an AI Classifier for Hatespeach Detection.\nYou are known for your rigourous and unpartial judgement and have vast experience in the field of hate speach. \n\nYou will be given a sentence from a forum and have to answer the question if this is hate speach.\n\nPlease exclusively respond in the following form to the question:\n```json\n{ \n  "relevant_facts": [\n    "fact_1",\n    "fact_2",\n    "fact_3",\n  ],\n  "contained_explicit_hate_speech_vocabular": ["word1", "word2", "word3", "word4"],\n  "argument_for_hate_speech": "short argument why this is hate speach",\n  "argument_against_hate_speech": "short argument why this is not hate speach",\n  "likelihood_of_presence_of_implicit_hate_speech": 0.17,\n  "likelihood_of_presence_of_explicit_hate_speech": 0.79,\n  "likelihood_of_ad_hominem_attack"; 013,\n  "likelihood_of_minority_attack": 0.24,\n  "like